Задание:
Используя данные из train.csv, построить
модель для предсказания цен на недвижимость (квартиры).
С помощью полученной модели предсказать
цены для квартир из файла test.csv.

Целевая переменная:
Price

Метрика:
R2 - коэффициент детерминации (sklearn.metrics.r2_score)

Сдача проекта:
1. Прислать в раздел Задания Урока 10 ("Вебинар. Консультация по итоговому проекту")
ссылку на программу в github (программа должна содержаться в файле Jupyter Notebook 
с расширением ipynb). (Pull request не нужен, только ссылка ведущая на сам скрипт).
2. Приложить файл с названием по образцу SShirkin_predictions.csv
с предсказанными ценами для квартир из test.csv (файл должен содержать два поля: Id, Price).
В файле с предсказаниями должна быть 5001 строка (шапка + 5000 предсказаний).

Сроки и условия сдачи:
Дедлайн: сдать проект нужно в течение 72 часов после начала Урока 10 ("Вебинар. Консультация по итоговому проекту").
Для успешной сдачи должны быть все предсказания (для 5000 квартир) и R2 должен быть больше 0.6.
При сдаче до дедлайна результат проекта может попасть в топ лучших результатов.
Повторная сдача и проверка результатов возможны только при условии предыдущей неуспешной сдачи.
Успешный проект нельзя пересдать в целях повышения результата.

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import seaborn as sns
from scipy.stats import norm, normaltest
#from sklearn.model_selection import train_test_split
from sklearn import set_config
import sys

In [3]:
df = pd.read_csv('/home/mikhail/Рабочий стол/Geek_Script_2605/Kurs_project_task/train.csv')
#df_tar = pd.read_csv('/home/mikhail/Рабочий стол/Geek_Script_2605/Kurs_project_task/train.csv')
# df = pd.read_csv('/home/mikhail/Рабочий стол/Geek_Script_2605/Geek_Home_DC/creditcard.csv')

In [4]:
df = pd.get_dummies(df)

In [5]:
# df=df.fillna(0.0).astype(int)
df = df.fillna(0.0)

In [6]:
feats = ['Id','DistrictId', 'Rooms', 'Square', 'KitchenSquare',
        'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3', 
        'Helthcare_2', 'Shops_1']

In [7]:

X = pd.DataFrame(df, columns=feats)

X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
dtypes: float64(5), int64(9)
memory usage: 1.1 MB


In [8]:
y = pd.DataFrame(df, columns=['Price'])
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
Price    10000 non-null float64
dtypes: float64(1)
memory usage: 78.2 KB


In [9]:
# X = df.loc[:,feats]
# y = df.loc[:, 'Price']

In [10]:
# X.info()

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [13]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=False)

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_valid_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

/home/mikhail/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/mikhail/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/mikhail/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


In [14]:
pca = PCA(random_state=42)

pca.fit(X_train_scaled)

print(pca.explained_variance_ratio_)

[0.2172884  0.12449276 0.108676   0.09603717 0.07369415 0.07164906
 0.06942753 0.06457634 0.05780649 0.03981853 0.03817908 0.0235137
 0.01230542 0.00253536]


In [15]:
pca.explained_variance_ratio_[:10].sum()# показывается процент сохраненной инофрмации

0.9234664520694773

In [16]:
pca = PCA(n_components=10, random_state=42)

X_train_mc = pca.fit_transform(X_train_scaled)
X_valid_mc = pca.transform(X_valid_scaled)

In [17]:
X_valid_mc

array([[-2.42650674,  0.38767685,  2.39180912, ..., -0.93322447,
         0.26669926, -0.10794269],
       [-2.36799911, -0.77121339,  0.71083689, ...,  0.83696045,
         0.18873   , -0.97213115],
       [ 1.43939654,  0.33642888, -1.11275703, ...,  0.02410621,
         1.1230479 , -0.00929034],
       ...,
       [ 4.91305179,  1.70073976,  0.0990906 , ...,  0.32877402,
         1.30798511, -0.35183093],
       [ 0.08299416,  0.43178931, -1.23372253, ..., -0.0889293 ,
        -0.53395453,  0.06245393],
       [-0.04448713, -0.18832941, -0.21979907, ..., -0.05136684,
         0.89369288,  0.39716821]])

In [18]:
# y_train=y_train.astype(int)
# y_train.info()

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression()

lr.fit(X_train_mc, y_train)

y_pred = lr.predict(X_valid_mc)

print(accuracy_score(y_test, y_pred))

/home/mikhail/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: Unknown label type: 'continuous'